In [1]:
import os
import sys

"""Linux"""
os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")
sys.path.append(os.path.abspath(os.path.join("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")))

os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast/libs/linux-gpu")


#from ctypes import FormatError
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os,sys

import torch
torch.use_deterministic_algorithms(True)
import pytorch_lightning as pl
from pytorch_forecasting.data.encoders import TorchNormalizer
from pytorch_forecasting.metrics import SMAPE, RMSE
from torchmetrics import R2Score, SymmetricMeanAbsolutePercentageError, MeanSquaredError

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import os,sys
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf,pacf
from scipy.signal import find_peaks
import operator
import statsmodels.api as sm
from itertools import combinations
import pickle
from pytorch_forecasting import Baseline
import random
from pytorch_forecasting import DeepAR,NegativeBinomialDistributionLoss
from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from optuna.trial import TrialState
import plotly
#from deepar_RegionWise_LinuxGpu_prediction_dev import train_and_forecast
from sklearn.metrics import classification_report

"""
Set Random seed
"""

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
## additional seeding to ensure reproduciblility.
pl.seed_everything(0)

2024-03-08 16:42:12.153376: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-08 16:42:12.231951: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 16:42:12.618692: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 16:42:12.618837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 16:42:12.690686: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

0

In [2]:
"""
Import pre-processed Data (tsfresh features)

response and target are the same thing
"""

### Your code
os.chdir('/home/optimusprime/Desktop/Data_Timothy')
############ Basically df = raw_feat_df <-------------------------
df = pd.read_parquet('EXP_4_combined_dd_ss_all_covariates.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,wind_speed,rain_1h,temp,clouds_all,snow_3h,rain_3h,humidity,city_name,weather_description,snow_1h
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,7.70,0.0,276.41,0,0.0,0.0,53,Alphabet City,sky is clear,0.0
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,8.23,0.0,275.35,0,0.0,0.0,56,Alphabet City,sky is clear,0.0
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,6.70,0.0,274.53,0,0.0,0.0,61,Alphabet City,sky is clear,0.0
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,6.20,0.0,274.13,0,0.0,0.0,63,Alphabet City,sky is clear,0.0
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,4.63,0.0,273.48,0,0.0,0.0,64,Alphabet City,sky is clear,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,4.63,0.0,282.29,20,0.0,0.0,50,Yorkville West,few clouds,0.0
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,4.12,0.0,283.77,20,0.0,0.0,47,Yorkville West,few clouds,0.0
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,3.60,0.0,283.52,75,0.0,0.0,50,Yorkville West,broken clouds,0.0
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,4.12,0.0,283.69,100,0.0,0.0,48,Yorkville West,overcast clouds,0.0


In [3]:
df = df.drop(columns=['city_name'])

In [4]:
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,wind_gust,wind_speed,rain_1h,temp,clouds_all,snow_3h,rain_3h,humidity,weather_description,snow_1h
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.00,7.70,0.0,276.41,0,0.0,0.0,53,sky is clear,0.0
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,9.77,8.23,0.0,275.35,0,0.0,0.0,56,sky is clear,0.0
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,0.00,6.70,0.0,274.53,0,0.0,0.0,61,sky is clear,0.0
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,10.30,6.20,0.0,274.13,0,0.0,0.0,63,sky is clear,0.0
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,0.00,4.63,0.0,273.48,0,0.0,0.0,64,sky is clear,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,0.00,4.63,0.0,282.29,20,0.0,0.0,50,few clouds,0.0
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,0.00,4.12,0.0,283.77,20,0.0,0.0,47,few clouds,0.0
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,0.00,3.60,0.0,283.52,75,0.0,0.0,50,broken clouds,0.0
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,0.00,4.12,0.0,283.69,100,0.0,0.0,48,overcast clouds,0.0


In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder_desc = LabelEncoder()
label_encoder_weather_main = LabelEncoder()

df['weather_desc_encoded'] = label_encoder_desc.fit_transform(df['weather_description'])
df['weather_main_encoded'] = label_encoder_weather_main.fit_transform(df['weather_main'])
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,rain_1h,temp,clouds_all,snow_3h,rain_3h,humidity,weather_description,snow_1h,weather_desc_encoded,weather_main_encoded
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,276.41,0,0.0,0.0,53,sky is clear,0.0,16,0
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,0.0,275.35,0,0.0,0.0,56,sky is clear,0.0,16,0
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,0.0,274.53,0,0.0,0.0,61,sky is clear,0.0,16,0
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,0.0,274.13,0,0.0,0.0,63,sky is clear,0.0,16,0
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,0.0,273.48,0,0.0,0.0,64,sky is clear,0.0,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,0.0,282.29,20,0.0,0.0,50,few clouds,0.0,2,1
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,0.0,283.77,20,0.0,0.0,47,few clouds,0.0,2,1
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,0.0,283.52,75,0.0,0.0,50,broken clouds,0.0,0,1
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,0.0,283.69,100,0.0,0.0,48,overcast clouds,0.0,14,1


In [ ]:
df = df.drop(columns=['weather_description', 'weather_main'])

In [ ]:
df.rename(columns={'weather_desc_encoded': 'weather_description', "weather_main_encoded":"weather_main"}, inplace=True)

In [ ]:
df['weather_description'] = df['weather_description'].astype(str)
df['weather_main'] = df['weather_main'].astype(str)

In [ ]:
os.chdir('/home/optimusprime/Desktop/Data_Timothy')
df.to_parquet('EXP_4_combined_dd_ss_all_covariates_encoded_further_preprocessed.parquet', engine='pyarrow')